In [8]:
import warnings
warnings.filterwarnings("ignore")

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import glob
import pickle
import platform
import copy
import umap

import proplot as pplt
import pandas as pd

from tqdm import tqdm
from vendi_score import vendi
from graph_utils import *
from data_utils import *
from model_utils import *
from analysis_utils import *
from saliency_utils import *
from generation_utils import *

DATA_DIR = '/scratch/gpfs/sj0161/topo_data/'
WEIGHT_DIR = '/scratch/gpfs/sj0161/topo_result/'
ANALYSIS_DIR = '/scratch/gpfs/sj0161/topo_analysis/'


pplt.rc['figure.facecolor'] = 'white'

COLORS = []
colors1 = pplt.Cycle('default')
colors2 = pplt.Cycle('538')

for color in colors1:
    COLORS.append(color['color'])

for color in colors2:
    COLORS.append(color['color'])

LATENT_DIM = 8

# Load Data

In [2]:
((x_train, y_train, c_train, l_train, graph_train),
(x_valid, y_valid, c_valid, l_valid, graph_valid),
(x_test, y_test, c_test, l_test, graph_test),
NAMES, SCALER, LE) = load_data(os.path.join(DATA_DIR, 'rg2.pickle'), fold=0, if_validation=True)

graph_all = np.concatenate((graph_train, graph_valid, graph_test))

# Hyperparameter Selection Based on Validation Dataset

In [6]:
# List of encoders to iterate through
encoders = ["desc_gnn", "gnn", "desc_dnn"]

results = []

for encoder in encoders:
    elbos, baccs, kls, cls, rls, rec, rmses, r2s, f1s, files_select = get_val_metrics(encoder=encoder)
    
    results.append([elbos, baccs, kls, cls, rls, rec, rmses, r2s, f1s, files_select])
    
with open(os.path.join(ANALYSIS_DIR, "hyper_select_result_all.pickle"), "wb") as handle:
    pickle.dump(results, handle)
    

Train: 858 Valid: 215 Test: 269


100%|██████████| 675/675 [07:42<00:00,  1.46it/s]


Train: 858 Valid: 215 Test: 269


100%|██████████| 675/675 [07:22<00:00,  1.52it/s]


Train: 858 Valid: 215 Test: 269


100%|██████████| 675/675 [04:48<00:00,  2.34it/s]


In [11]:
with open(os.path.join(ANALYSIS_DIR, "hyper_select_result_all.pickle"), "rb") as handle:
    results = pickle.load(handle)

# Concatenate arrays from all results
baccs = np.concatenate([result[1] for result in results])
r2s = np.concatenate([result[7] for result in results])
f1s = np.concatenate([result[8] for result in results])
kls = np.concatenate([result[2] for result in results])
rmses = np.concatenate([result[6] for result in results])
files_select = np.concatenate([result[9] for result in results])

# Define indices for separating results of different encoders
idx = [0, 
       len(results[0][0]), 
       len(results[0][0]) + len(results[0][0]), 
       len(results[0][0]) + len(results[1][0]) + len(results[2][0])]

# Calculate the Pareto front and find the best points
pareto_indices, pareto_front = pareto_frontier(1 - baccs, 1 - r2s, 1 - f1s, kls, limits=[0.1, 0.1, 0.1, np.inf])
          

In [18]:
data = pd.DataFrame(columns=[
    "Encoder", "File Name", "BACC", "RMSE", "R2", "F1", "KL", "Distance to Origin"
])

for i, encoder in enumerate(encoders):
    
    idx_temp = np.where((np.array(pareto_indices) >= idx[i]) & (np.array(pareto_indices) < idx[i+1]))[0]
    
    pareto_front_temp = pareto_front[idx_temp]
    best_point, best_idx = closest_to_origin(pareto_front_temp)#, vmin, vmax)

    # Extract information for the best point
    encoder_name = encoder
    file_name = files_select[best_idx].split('/')[-1].split('.pickle')[0]
    bacc = baccs[best_idx]
    rmse = rmses[best_idx]
    r2 = r2s[best_idx]
    f1 = f1s[best_idx]
    kl = kls[best_idx]
    distance_to_origin = np.linalg.norm(np.array(best_point[:]))

    # Create a dictionary with the data for the current encoder
    row_data = {
        "Encoder": encoder_name,
        "File Name": file_name,
        "BACC": bacc,
        "RMSE": rmse,
        "R2": r2,
        "F1": f1,
        "KL": kl,
        "Distance to Origin": distance_to_origin
    }

    # Append the data to the DataFrame
    data = data.append(row_data, ignore_index=True)

data.to_csv(os.path.join(ANALYSIS_DIR, "hyper_select_result.csv"), index=False)

In [19]:
data

,Encoder,File Name,BACC,RMSE,R2,F1,KL,Distance to Origin
0,desc_gnn,desc_gnn_cnn_20230828_8_val_decoder_acc_True_T...,0.943852,1.467548,0.991504,0.995348,18.724371,0.382945
1,gnn,gnn_cnn_20230828_8_val_decoder_acc_True_True_1...,0.944785,3.044410,0.963435,0.976836,15.601805,0.842681
2,desc_dnn,desc_dnn_cnn_20230828_8_val_decoder_acc_True_T...,0.928105,1.140703,0.994867,0.995348,16.041761,0.399231


# Results of the Best Model of Each Encoder Type

### With 10 Repetitions Considering the Randomness of Sampling Layer

In [20]:
files = [
    "desc_gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 1]_0.001_64",
    "gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 100]_0.01_64",
    "desc_dnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 0.01]_0.01_64",
]

baccs = []
rmses = []
r2s = []
f1s = []
accs = []
train_outs = []
valid_outs = []
test_outs = []

for file in files:
    ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

    model, pickle_file = train_vae(ENCODER, DECODER, MONITOR,
                  IF_REG, IF_CLS,
                  x_train, x_valid,
                  y_train, y_valid,
                  c_train, c_valid,
                  l_train, l_valid,
                  1.0, weights,
                  LR, BS,
                  False, ) 

    train_out, valid_out, test_out, bacc, rmse, r2, f1, acc = get_metrics(model, ENCODER, IF_REG, IF_CLS,
                                                               x_train, y_train, c_train, l_train,
                                                               x_valid, y_valid, c_valid, l_valid,
                                                               x_test, y_test, c_test, l_test,
                                                               n_repeat=10, if_bacc=True)
    baccs.append(bacc)
    rmses.append(rmse)
    r2s.append(r2)
    f1s.append(f1)
    accs.append(acc)
    train_outs.append(train_out)
    valid_outs.append(valid_out)
    test_outs.append(test_out)
    
baccs = np.array(baccs)
rmses = np.array(rmses)
r2s = np.array(r2s)
f1s = np.array(f1s)
accs = np.array(accs)


with open(os.path.join(ANALYSIS_DIR, "accuracy_metric.pickle"), "wb") as handle:
    pickle.dump(baccs, handle)
    pickle.dump(rmses, handle)
    pickle.dump(r2s, handle)
    pickle.dump(f1s, handle)
    pickle.dump(accs, handle)

with open(os.path.join(ANALYSIS_DIR, "all_outs.pickle"), "wb") as handle:
    pickle.dump(train_outs, handle)
    pickle.dump(valid_outs, handle)
    pickle.dump(test_outs, handle)

RMSE: Train 1.19+/-0.03 Valid 1.47+/-0.05 Test 1.49+/-0.06
R2:   Train 0.99+/-0.00 Valid 0.99+/-0.00 Test 0.99+/-0.00
F1:   Train 1.00+/-0.00 Valid 0.98+/-0.01 Test 0.96+/-0.01
BACC: Train 1.00+/-0.00 Valid 0.94+/-0.00 Test 0.94+/-0.00
RMSE: Train 2.38+/-0.05 Valid 3.16+/-0.06 Test 3.14+/-0.07
R2:   Train 0.98+/-0.00 Valid 0.96+/-0.00 Test 0.96+/-0.00
F1:   Train 1.00+/-0.00 Valid 0.97+/-0.00 Test 0.98+/-0.00
BACC: Train 0.99+/-0.00 Valid 0.94+/-0.00 Test 0.94+/-0.00
RMSE: Train 1.05+/-0.03 Valid 1.27+/-0.10 Test 1.99+/-0.06
R2:   Train 1.00+/-0.00 Valid 0.99+/-0.00 Test 0.98+/-0.00
F1:   Train 1.00+/-0.00 Valid 1.00+/-0.00 Test 0.97+/-0.00
BACC: Train 0.97+/-0.00 Valid 0.92+/-0.00 Test 0.92+/-0.00


### With 1 Repetition for Sample Test Set Prediction

In [22]:
files = [
    "desc_gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 1]_0.001_64",
    "gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 100]_0.01_64",
    "desc_dnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 0.01]_0.01_64",
]

test_outs = []

for file in files:
    ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

    model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                                   x_train, x_valid, y_train, y_valid, c_train, c_valid,
                                   l_train, l_valid, 1.0, weights, LR, BS, False) 

    train_out, valid_out, test_out, bacc, rmse, r2, f1, acc = get_metrics(model, ENCODER, IF_REG, IF_CLS,
                                                               x_train, y_train, c_train, l_train,
                                                               x_valid, y_valid, c_valid, l_valid,
                                                               x_test, y_test, c_test, l_test,
                                                               n_repeat=1,if_bacc=True)
    test_outs.append(test_out)

with open(os.path.join(ANALYSIS_DIR, "test_out.pickle"), "wb") as handle:
    pickle.dump(test_outs, handle)

RMSE: Train 1.16+/-0.00 Valid 1.56+/-0.00 Test 1.50+/-0.00
R2:   Train 1.00+/-0.00 Valid 0.99+/-0.00 Test 0.99+/-0.00
F1:   Train 1.00+/-0.00 Valid 0.99+/-0.00 Test 0.97+/-0.00
BACC: Train 1.00+/-0.00 Valid 0.94+/-0.00 Test 0.94+/-0.00
RMSE: Train 2.42+/-0.00 Valid 3.21+/-0.00 Test 3.26+/-0.00
R2:   Train 0.98+/-0.00 Valid 0.96+/-0.00 Test 0.96+/-0.00
F1:   Train 1.00+/-0.00 Valid 0.97+/-0.00 Test 0.97+/-0.00
BACC: Train 0.99+/-0.00 Valid 0.94+/-0.00 Test 0.94+/-0.00
RMSE: Train 1.06+/-0.00 Valid 1.22+/-0.00 Test 1.91+/-0.00
R2:   Train 1.00+/-0.00 Valid 0.99+/-0.00 Test 0.99+/-0.00
F1:   Train 1.00+/-0.00 Valid 1.00+/-0.00 Test 0.97+/-0.00
BACC: Train 0.97+/-0.00 Valid 0.92+/-0.00 Test 0.92+/-0.00


# Saliency Map Calculation

In [24]:
files = [
    "desc_gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 1]_0.001_64",
    "desc_dnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 0.01]_0.01_64",
]

grads = []

for file in files:
    ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

    model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                                   x_train, x_valid, y_train, y_valid, c_train, c_valid,
                                   l_train, l_valid, 1.0, weights, LR, BS, False) 

    train_out, valid_out, test_out, bacc, rmse, r2, f1, acc = get_metrics(model, ENCODER, IF_REG, IF_CLS,
                                                               x_train, y_train, c_train, l_train,
                                                               x_valid, y_valid, c_valid, l_valid,
                                                               x_test, y_test, c_test, l_test,
                                                               n_repeat=1, if_bacc=False)

    grad = compute_saliency(model, x_train, y_train, l_train, c_train, 
                             output_index=1, enc_type=ENCODER, if_reg=IF_REG, if_cls=IF_CLS)
    
    grads.append(grad)

with open(os.path.join(ANALYSIS_DIR, "saliency.pickle"), "wb") as handle:
    pickle.dump(grads, handle)
    pickle.dump(files, handle)

RMSE: Train 1.16+/-0.00 Valid 1.56+/-0.00 Test 1.50+/-0.00
R2:   Train 1.00+/-0.00 Valid 0.99+/-0.00 Test 0.99+/-0.00
F1:   Train 1.00+/-0.00 Valid 0.99+/-0.00 Test 0.97+/-0.00
RMSE: Train 1.06+/-0.00 Valid 1.22+/-0.00 Test 1.91+/-0.00
R2:   Train 1.00+/-0.00 Valid 0.99+/-0.00 Test 0.99+/-0.00
F1:   Train 1.00+/-0.00 Valid 1.00+/-0.00 Test 0.97+/-0.00


# Latent Space Calculation

### Different Encoders

In [25]:
files = [
    "desc_gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 1]_0.001_64",
    "gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 100]_0.01_64",
    "desc_dnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 0.01]_0.01_64",
]

for file in files:
    ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

    model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                                   x_train, x_valid, y_train, y_valid, c_train, c_valid,
                                   l_train, l_valid, 1.0, weights, LR, BS, False) 

    latent_train = latent_model(model, data=[x_train, l_train], enc_type=ENCODER, mean_var=False)
    latent_valid = latent_model(model, data=[x_valid, l_valid], enc_type=ENCODER, mean_var=False)
    latent_test = latent_model(model, data=[x_test, l_test], enc_type=ENCODER, mean_var=False)
    
    short_name = file.split("_20230828")[0]
    
    if short_name == "desc_gnn_cnn":
        short_name = ""
    elif short_name == "gnn_cnn":
        short_name = "_gnn"
    elif short_name == "desc_dnn_cnn":
        short_name = "_topo"

    with open(os.path.join(ANALYSIS_DIR, f"latent_space{short_name}.pickle"), "wb") as handle:
        pickle.dump(latent_train, handle)
        pickle.dump(latent_valid, handle)
        pickle.dump(latent_test, handle)

### Auxilary Tasks

In [26]:
files = ["desc_gnn_cnn_20230828_8_val_loss_False_True_1.0_[1.0, 1]_0.001_64",
        "desc_gnn_cnn_20230828_8_val_loss_True_False_1.0_[1.0, 1]_0.001_64",
        "desc_gnn_cnn_20230828_8_val_loss_False_False_1.0_[1.0]_0.001_64",
       
]


for file in files:
    ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

    model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                                   x_train, x_valid, y_train, y_valid, c_train, c_valid,
                                   l_train, l_valid, 1.0, weights, LR, BS, False) 

    latent_train = latent_model(model, data=[x_train, l_train], enc_type=ENCODER, mean_var=False)
    latent_valid = latent_model(model, data=[x_valid, l_valid], enc_type=ENCODER, mean_var=False)
    latent_test = latent_model(model, data=[x_test, l_test], enc_type=ENCODER, mean_var=False)
    
    short_name = "_ref_" + file.split("_")[-6].lower() + "_cls_" + file.split("_")[-5].lower()

    with open(os.path.join(ANALYSIS_DIR, f"latent_space{short_name}.pickle"), "wb") as handle:
        pickle.dump(latent_train, handle)
        pickle.dump(latent_valid, handle)
        pickle.dump(latent_test, handle)

# UMAP Generation

### UMAP TopoGNN

In [12]:
# Load model
file = "desc_gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 1]_0.001_64"

ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                               x_train, x_valid, y_train, y_valid, c_train, c_valid,
                               l_train, l_valid, 1.0, weights, LR, BS, False) 

# Load latent space
with open(os.path.join(ANALYSIS_DIR, "latent_space.pickle"), "rb") as handle:
    latent_train = pickle.load(handle)
    latent_valid = pickle.load(handle)
    latent_test = pickle.load(handle)
    
latent_all = np.concatenate((latent_train, latent_valid, latent_test), axis=0)

# UMAP transformation
u = umap.UMAP(n_components=2, random_state=0)
z = u.fit_transform(latent_all)
            
with open(os.path.join(ANALYSIS_DIR, "umap.pickle"), "wb") as handle:
    pickle.dump(z, handle)
    pickle.dump(u, handle)

### UMAP GNN

In [13]:
# Load model
file = "gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 100]_0.01_64"

ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                               x_train, x_valid, y_train, y_valid, c_train, c_valid,
                               l_train, l_valid, 1.0, weights, LR, BS, False) 

# Load latent space
with open(os.path.join(ANALYSIS_DIR, "latent_space_gnn.pickle"), "rb") as handle:
    latent_train = pickle.load(handle)
    latent_valid = pickle.load(handle)
    latent_test = pickle.load(handle)
    
latent_all = np.concatenate((latent_train, latent_valid, latent_test), axis=0)

# UMAP transformation
u = umap.UMAP(n_components=2, random_state=0)
z = u.fit_transform(latent_all)
            
with open(os.path.join(ANALYSIS_DIR, "umap_gnn.pickle"), "wb") as handle:
    pickle.dump(z, handle)
    pickle.dump(u, handle)

### UMAP Topo

In [ ]:
# Load model
file = "desc_dnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 0.01]_0.01_64"

ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                               x_train, x_valid, y_train, y_valid, c_train, c_valid,
                               l_train, l_valid, 1.0, weights, LR, BS, False) 

# Load latent space
with open(os.path.join(ANALYSIS_DIR, "latent_space_topo.pickle"), "rb") as handle:
    latent_train = pickle.load(handle)
    latent_valid = pickle.load(handle)
    latent_test = pickle.load(handle)
    
latent_all = np.concatenate((latent_train, latent_valid, latent_test), axis=0)

# UMAP transformation
u = umap.UMAP(n_components=2, random_state=0)
z = u.fit_transform(latent_all)
            
with open(os.path.join(ANALYSIS_DIR, "umap_topo.pickle"), "wb") as handle:
    pickle.dump(z, handle)
    pickle.dump(u, handle)

### UMAP No Classsification No Regression

In [15]:
# Load model
file = "desc_gnn_cnn_20230828_8_val_loss_False_False_1.0_[1.0]_0.001_64"

ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                               x_train, x_valid, y_train, y_valid, c_train, c_valid,
                               l_train, l_valid, 1.0, weights, LR, BS, False) 

# Load latent space
with open(os.path.join(ANALYSIS_DIR, "latent_space_reg_false_cls_false.pickle"), "rb") as handle:
    latent_train = pickle.load(handle)
    latent_valid = pickle.load(handle)
    latent_test = pickle.load(handle)
    
latent_all = np.concatenate((latent_train, latent_valid, latent_test), axis=0)

# UMAP transformation
u = umap.UMAP(n_components=2, random_state=0)
z = u.fit_transform(latent_all)
            
with open(os.path.join(ANALYSIS_DIR, "umap_reg_false_cls_false.pickle"), "wb") as handle:
    pickle.dump(z, handle)
    pickle.dump(u, handle)

### UMAP No Classification Only

In [16]:
# Load model
file = "desc_gnn_cnn_20230828_8_val_loss_True_False_1.0_[1.0, 1]_0.001_64"

ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                               x_train, x_valid, y_train, y_valid, c_train, c_valid,
                               l_train, l_valid, 1.0, weights, LR, BS, False) 

# Load latent space
with open(os.path.join(ANALYSIS_DIR, "latent_space_reg_true_cls_false.pickle"), "rb") as handle:
    latent_train = pickle.load(handle)
    latent_valid = pickle.load(handle)
    latent_test = pickle.load(handle)
    
latent_all = np.concatenate((latent_train, latent_valid, latent_test), axis=0)

# UMAP transformation
u = umap.UMAP(n_components=2, random_state=0)
z = u.fit_transform(latent_all)
            
with open(os.path.join(ANALYSIS_DIR, "umap_reg_true_cls_false.pickle"), "wb") as handle:
    pickle.dump(z, handle)
    pickle.dump(u, handle)

### UMAP No Regression Only

In [17]:
# Load model
file = "desc_gnn_cnn_20230828_8_val_loss_False_False_1.0_[1.0]_0.001_64"

ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                               x_train, x_valid, y_train, y_valid, c_train, c_valid,
                               l_train, l_valid, 1.0, weights, LR, BS, False) 

# Load latent space
with open(os.path.join(ANALYSIS_DIR, "latent_space_reg_false_cls_true.pickle"), "rb") as handle:
    latent_train = pickle.load(handle)
    latent_valid = pickle.load(handle)
    latent_test = pickle.load(handle)
    
latent_all = np.concatenate((latent_train, latent_valid, latent_test), axis=0)

# UMAP transformation
u = umap.UMAP(n_components=2, random_state=0)
z = u.fit_transform(latent_all)
            
with open(os.path.join(ANALYSIS_DIR, "umap_reg_false_cls_true.pickle"), "wb") as handle:
    pickle.dump(z, handle)
    pickle.dump(u, handle)

# Polymer Topology Generation Using TopoGNN Latent Space UMAP

### TopoGNN Fix Z1, Increase Z2.

In [19]:
def interpolate(start, end, num_points):
    """
    Interpolate between two points in n-dimensional space.

    Parameters:
        start (array-like): The starting point as an array-like object.
        end (array-like): The ending point as an array-like object.
        num_points (int): The number of points to interpolate between start and end.

    Returns:
        np.ndarray: An array containing the interpolated points between start and end.
    """
    start = np.array(start)
    end = np.array(end)
    t_values = np.linspace(0, 1, num_points)
    points = [(1-t)*start + t*end for t in t_values]
    return np.array(points)



file = "desc_gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 1]_0.001_64"

ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                               x_train, x_valid, y_train, y_valid, c_train, c_valid,
                               l_train, l_valid, 1.0, weights, LR, BS, False) 
            
with open(os.path.join(ANALYSIS_DIR, "umap.pickle"), "rb") as handle:
    _ = pickle.load(handle)
    u = pickle.load(handle)

In [10]:
start = np.array([1, 4])
end   = np.array([1, 16])
num_points = 20
max_iter = 50

points = interpolate(start, end, num_points)

outputs = []

for i in range(num_points):
    point = points[i]
    
    for _ in range(max_iter):
        point_ = point + np.random.normal(0, 0.2, size=(2,)) # random pertubation in a neighborhood
        data = u.inverse_transform(point_[None, ...])
        
        G0, G, gen_cls, cln_cls, gen_reg, cln_reg_m, cln_reg_s = polymer_generation(model, data, None, ENCODER)
        
        if check_valid(gen_reg, cln_reg_m, gen_cls, cln_cls):
            outputs.append([data, point_, G0, G, gen_cls, cln_cls, gen_reg, cln_reg_m, cln_reg_s])
            break
            
with open(os.path.join(ANALYSIS_DIR, "umap_1_4_1_16.pickle"), "wb") as handle:
    pickle.dump(outputs, handle)

### TopoGNN Fix Z2, Increase Z1.

In [11]:
start = np.array([-3, 12])
end   = np.array([6, 12])
num_points = 20
max_iter = 50

points = interpolate(start, end, num_points)

outputs = []

for i in range(num_points):
    point = points[i]
    
    for _ in range(max_iter):
        point_ = point + np.random.normal(0, 0.2, size=(2,))
        data = u.inverse_transform(point_[None, ...])
        
        G0, G, gen_cls, cln_cls, gen_reg, cln_reg_m, cln_reg_s = polymer_generation(model, data, None, ENCODER)
        
        if check_valid(gen_reg, cln_reg_m, gen_cls, cln_cls):
            outputs.append([data, point_, G0, G, gen_cls, cln_cls, gen_reg, cln_reg_m, cln_reg_s])
            break
            
with open(os.path.join(ANALYSIS_DIR, "umap_-3_12_6_12.pickle"), "wb") as handle:
    pickle.dump(outputs, handle)

# Property Guided Topology Generation

In [23]:
def check_isomorphism(graph_list, new_graph):
    for graph in graph_list:
        if nx.is_isomorphic(graph, new_graph):
            return True
    return False

def rg_latent_vector(l, y_train, c_train, poly_type='branch', target_rg=40):
    
    idx = np.where(NAMES == poly_type)[0][0]

    a = l[np.where(c_train == idx)[0]]
    y = y_train[np.where(c_train == idx)[0]]
    
    if np.abs(y - target_rg).min() < 1:
        
        idx2 = np.where(np.abs(y - target_rg) < 1)[0]

        return a[idx2], y[idx2]
    
    else:
        
        return None, None

In [31]:
# Load latent space
with open(os.path.join(ANALYSIS_DIR, "latent_space.pickle"), "rb") as handle:
    latent_train = pickle.load(handle)
    latent_valid = pickle.load(handle)
    latent_test = pickle.load(handle)
    
latent_all = np.concatenate((latent_train, latent_valid, latent_test), axis=0)

# Load data label
((x_train, y_train, c_train, l_train, graph_train),
(x_valid, y_valid, c_valid, l_valid, graph_valid),
(x_test, y_test, c_test, l_test, graph_test),
NAMES, SCALER, LE) = load_data(os.path.join(DATA_DIR, 'rg2.pickle'), fold=0, if_validation=True)

graph_all = np.concatenate((graph_train, graph_valid, graph_test))
y_all = np.concatenate((y_train, y_valid, y_test))
c_all = np.concatenate((c_train, c_valid, c_test))

# Load TopoGNN model
file = "desc_gnn_cnn_20230828_8_val_decoder_acc_True_True_1.0_[1.0, 1, 1]_0.001_64"

ENCODER, DECODER, MONITOR, IF_REG, IF_CLS, weights, LR, BS = get_spec(file)

model, pickle_file = train_vae(ENCODER, DECODER, MONITOR, IF_REG, IF_CLS,
                               x_train, x_valid, y_train, y_valid, c_train, c_valid,
                               l_train, l_valid, 1.0, weights, LR, BS, False)

In [38]:
def gen_prop_polymer(target_rg=30, target_top="branch", max_iter=1000):
    outputs = []
    graphs = []

    latent_vector, rg_dataset = rg_latent_vector(latent_all, y_all, c_all, target_top, target_rg)
    
    if latent_vector is None:
        raise Exception("The target rg2 is too large/small ")

    for i in range(max_iter):
        noise = np.random.normal(0, 1, (1, 8)) * 0.1
        num = len(latent_vector)

        for j in range(num):
            K.clear_session()
            d_in = latent_vector[j, ...] + noise
            graph_raw, graph, gen_cls, cln_cls, gen_reg, cln_reg_m, cln_reg_s = polymer_generation(model, d_in, None, ENCODER)

            flag1 = np.abs(gen_reg - cln_reg_m) < 2
            flag2 = np.abs(gen_reg - target_rg) < 2
            flag3 = np.abs(cln_reg_m - target_rg) < 2
            flag4 = gen_cls == cln_cls
            flag5 = cln_cls == target_top

            x_clean_ = nx.to_numpy_array(graph)
            n_clean = len(x_clean_)
            x_clean = np.zeros((1, 100, 100))
            x_clean[0, :n_clean, :n_clean] = x_clean_
            x_clean = x_clean.astype("int")

            l_clean = get_desc(graph)[None, ...]
            l_clean = SCALER.transform(l_clean)

            d_clean = latent_model(model, data=[x_clean, l_clean], enc_type=ENCODER, mean_var=False)

            if flag1 and flag2 and flag3 and flag4 and flag5: 
                if len(graphs) > 0:
                    if not check_isomorphism(graphs, graph) and not check_isomorphism(graph_all, graph):
                        graphs.append(graph)
                        outputs.append([d_in, graph_raw, graph, gen_cls, cln_cls, gen_reg, cln_reg_m, cln_reg_s, latent_vector[j, ...], d_in, d_clean])
                else:
                    graphs.append(graph)
                    outputs.append([d_in, graph_raw, graph, gen_cls, cln_cls, gen_reg, cln_reg_m, cln_reg_s, latent_vector[j, ...], d_in, d_clean])

    
    # check latent space distance
    z_cleans = []
    z_raws   = []
    rmses    = []
    new_outputs = []
    
    for i in range(len(outputs)):
        graph = outputs[i][2]
        x_clean_ = nx.to_numpy_array(graph)
        n_clean = len(x_clean_)
        x_clean = np.zeros((1, 100, 100))
        x_clean[0, :n_clean, :n_clean] = x_clean_
        x_clean = x_clean.astype("int")

        l_clean = get_desc(graph)[None, ...]
        l_clean = SCALER.transform(l_clean)

        z_clean = latent_model(model, data=[x_clean, l_clean], enc_type=ENCODER, mean_var=False).squeeze()

        z_raw = outputs[i][0].squeeze()
        rmse = skm.mean_absolute_error(z_raw, z_clean)

        if rmse < 1:
            z_cleans.append(z_clean)
            z_raws.append(z_raw)
            rmses.append(rmse)
            new_outputs.append(outputs[i]+[z_clean])
            
    return new_outputs

In [45]:
target_rg = 30

for target_top in ["star", "branch", "comb", "cyclic"]:
    new_outputs = gen_prop_polymer(target_rg=target_rg, target_top=target_top, max_iter=1000)

    with open(os.path.join(ANALYSIS_DIR, f"gen_{target_top}_{target_rg}_all.pickle"), "wb") as handle:
        pickle.dump(new_outputs, handle)

In [46]:
target_rg = 50

for target_top in ["branch", "comb"]:
    new_outputs = gen_prop_polymer(target_rg=target_rg, target_top=target_top, max_iter=1000)

    with open(os.path.join(ANALYSIS_DIR, f"gen_{target_top}_{target_rg}_all.pickle"), "wb") as handle:
        pickle.dump(new_outputs, handle)

In [47]:
target_rg = 7.5

for target_top in ["star", "dendrimer"]:
    new_outputs = gen_prop_polymer(target_rg=target_rg, target_top=target_top, max_iter=1000)

    with open(os.path.join(ANALYSIS_DIR, f"gen_{target_top}_{target_rg}_all.pickle"), "wb") as handle:
        pickle.dump(new_outputs, handle)

# Diversity Calculation Using Vendi Score

In [10]:
# convert all graphs into graph eigen spectra
graph_total = [graph_train, graph_valid, graph_test]

lap_spec_data = []

for graphs in graph_total:
    for G in graphs:
        lap_spec = nx.laplacian_spectrum(G)
        lap_spec_zero_pad = np.zeros((100,))
        lap_spec_zero_pad[:len(lap_spec)] = lap_spec
        lap_spec_data.append(lap_spec_zero_pad)
        
lap_spec_data = np.array(lap_spec_data)

with open(os.path.join(ANALYSIS_DIR, "lap_spec_data.pickle"), "wb") as handle:
    pickle.dump(lap_spec_data, handle)

### Dataset Vendi Score

In [4]:
print(f"Dataset Vendi Score: {vendi.score_dual(lap_spec_data):0.4f}")

Dataset Vendi Score: 2.0968


### Vendi Score for the Latent Space

In [7]:
files = [
    "latent_space_desc_gnn_cnn.pickle",
    "latent_space_gnn_cnn.pickle",
    "latent_space_desc_dnn_cnn.pickle"
]

for file in files:
    with open(os.path.join(ANALYSIS_DIR, file), "rb") as handle:
        latent_data = pickle.load(handle)
    print(file.split(".pickle")[0])
    print(f"Dataset Vendi Score: {vendi.score_dual(latent_data):0.4f} \n")

latent_space_desc_gnn_cnn
Dataset Vendi Score: 7.3225 

latent_space_gnn_cnn
Dataset Vendi Score: 7.4370 

latent_space_desc_dnn_cnn
Dataset Vendi Score: 7.0863 



In [8]:
files = [
    "latent_space_False_False.pickle",
    "latent_space_False_True.pickle",
    "latent_space_True_False.pickle"
]

for file in files:
    with open(os.path.join(ANALYSIS_DIR, file), "rb") as handle:
        latent_data = pickle.load(handle)
        print(file.split(".pickle")[0])
    print(f"Dataset Vendi Score: {vendi.score_dual(latent_data):0.4f} \n")

latent_space_False_False
Dataset Vendi Score: 5.8532 

latent_space_False_True
Dataset Vendi Score: 6.3171 

latent_space_True_False
Dataset Vendi Score: 5.3128 



### Vendi Score for Random Generation Based on Different Models

In [13]:
with open(os.path.join(ANALYSIS_DIR, "no_valid_random_gen_desc_gnn_cnn.pickle"), "rb") as handle:
    gen_data = pickle.load(handle)
    
gen_clean_graph = [gen_data[i][2] for i in range(len(gen_data))]

lap_spec_data = []

for G in gen_clean_graph:
    lap_spec = nx.laplacian_spectrum(G)
    lap_spec_zero_pad = np.zeros((100,))
    lap_spec_zero_pad[:len(lap_spec)] = lap_spec
    lap_spec_data.append(lap_spec_zero_pad)

print(f"Dataset Vendi Score: {vendi.score_dual(lap_spec_data):0.4f}")

Dataset Vendi Score: 5.0684


In [14]:
with open(os.path.join(ANALYSIS_DIR, "no_valid_random_gen_gnn_cnn.pickle"), "rb") as handle:
    gen_data = pickle.load(handle)
    
gen_clean_graph = [gen_data[i][2] for i in range(len(gen_data))]

lap_spec_data = []

for G in gen_clean_graph:
    lap_spec = nx.laplacian_spectrum(G)
    lap_spec_zero_pad = np.zeros((100,))
    lap_spec_zero_pad[:len(lap_spec)] = lap_spec
    lap_spec_data.append(lap_spec_zero_pad)

print(f"Dataset Vendi Score: {vendi.score_dual(lap_spec_data):0.4f}")

Dataset Vendi Score: 4.9580


In [16]:
with open(os.path.join(ANALYSIS_DIR, "no_valid_random_gen_desc_dnn_cnn.pickle"), "rb") as handle:
    gen_data = pickle.load(handle)
    
gen_clean_graph = [gen_data[i][2] for i in range(len(gen_data))]

lap_spec_data = []

for G in gen_clean_graph:
    lap_spec = nx.laplacian_spectrum(G)
    lap_spec_zero_pad = np.zeros((100,))
    lap_spec_zero_pad[:len(lap_spec)] = lap_spec
    lap_spec_data.append(lap_spec_zero_pad)

print(f"Dataset Vendi Score: {vendi.score_dual(lap_spec_data):0.4f}")

Dataset Vendi Score: 4.3305
